# Extract data for urban calculations

Test input for Tanzania

0. Select focal ADM, buffer by 1km, rasterize as [0, 1]
1. Download DEM data from ASTER, mosaick
2. Calculate slope of DEM
3. Extract water layer from Globcover
4. Rasterize building footprints
5. Select population layer
6. Standardize all rasters to population layer  
   a. Set area outside of focal admin to NoData  
   b. Set everything to 16bit  
   
   


In [1]:
import sys, os, importlib, shutil
import requests
import rasterio, elevation, richdem
import rasterio.warp
from rasterio import features

import pandas as pd
import geopandas as gpd
import numpy as np

from shapely.geometry import MultiPolygon, Polygon

#Import GOST urban functions
sys.path.append("../")
import src.UrbanRaster as urban
import src.urban_helper as helper

#Import raster helpers
sys.path.append("../../gostrocks/src")

import GOSTRocks.rasterMisc as rMisc
from GOSTRocks.misc import tPrint

In [2]:
global_bounds = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
global_bounds_adm2 = "/home/public/Data/GLOBAL/ADMIN/Admin2_Polys.shp"
global_landcover  = "/home/public/Data/GLOBAL/LANDCOVER/GLOBCOVER/2015/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7.tif"
global_ghspop = "/home/public/Data/GLOBAL/Population/GHS/250/GHS_POP_E2015_GLOBE_R2019A_54009_250_V1_0.tif"
global_ghbuilt = "/home/public/Data/GLOBAL/URBAN/GHS/GHS_1K_BUILT/GHS_BUILT_LDS2014_GLOBE_R2018A_54009_1K_V1_0.tif"
inG = gpd.read_file(global_bounds)
inG2 = gpd.read_file(global_bounds_adm2)
iso3 = 'GHA'
output_folder = "/home/wb411133/temp/%s_URBAN_DATA" % iso3

#Define paths to local population files
ea_file = os.path.join(output_folder, "FINAL_EAS.shp")
inEA = gpd.read_file(ea_file)
local_path = "/home/public/Data/COUNTRY/{country}/POPULATION/WORLDPOP/".format(country=iso3)
pop_2015_con = os.path.join(local_path, "ppp_prj_2015_%s_UNadj.tif" % iso3)
pop_2018_con = os.path.join(local_path, "ppp_prj_2017_%s_UNadj.tif" % iso3)
pop_2015_un = os.path.join(local_path, "%s_ppp_2015_UNadj.tif" % iso3.lower())
pop_2018_un = os.path.join(local_path, "%s_ppp_2017_UNadj.tif" % iso3.lower())
pop_files = [pop_2015_un, pop_2018_un, pop_2015_con, pop_2018_con]

inD = inG.loc[inG['ISO3'] == iso3]
inD['geometry'] = inD['geometry'].apply(lambda x: x.buffer(500))
inD = inD.to_crs({'init':'epsg:4326'})

inD2 = inG2.loc[inG2['ISO3'] == iso3]
inD2 = inD2.to_crs({'init':'epsg:4326'})

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
bad_shapes = inEA['geometry'].apply(lambda x: x is None)
inEA = inEA.loc[~bad_shapes]

In [9]:
importlib.reload(helper)
importlib.reload(rMisc)

xx = helper.urban_country(iso3, output_folder, inD)
tPrint("***** Extracting Global Layers %s" % iso3)
xx.extract_layers(global_landcover, global_ghspop, global_ghbuilt)
tPrint("***** Standardizing rasters")
xx.standardize_rasters(pop_files=pop_files)
tPrint("***** Calculating Urban")
xx.calculate_urban(pop_files)
xx.calculate_urban([xx.ghspop_file])
tPrint("***** Calculating Zonal admin2")
if not os.path.exists(os.path.join(output_folder, "URBAN_ADMIN2_STATS_COMPILED.csv")):
    zonal_adm2 = xx.pop_zonal_admin(inD2)
    zonal_adm2.to_csv(os.path.join(output_folder, "URBAN_ADMIN2_STATS_COMPILED.csv"))
tPrint("***** Calculating Zonal communes")
if not os.path.exists(os.path.join(output_folder, "URBAN_COMMUNE_STATS_COMPILED.csv")):
    zonal_ea = xx.pop_zonal_admin(inEA)
    zonal_ea.to_csv(os.path.join(output_folder, "URBAN_COMMUNE_STATS_COMPILED.csv"))

11:43:01	***** Extracting Global Layers GHA
11:43:01	***** Standardizing rasters
/home/wb411133/temp/GHA_URBAN_DATA/GHA_ADMIN.tif
/home/wb411133/temp/GHA_URBAN_DATA/GHA_GHS.tif
/home/wb411133/temp/GHA_URBAN_DATA/GHA_LC_H20.tif
/home/wb411133/temp/GHA_URBAN_DATA/GHA_SLOPE.tif
/home/wb411133/temp/GHA_URBAN_DATA/GHA_DEM.tif
/home/wb411133/temp/GHA_URBAN_DATA/GHA_GHSBUILT.tif
/home/public/Data/COUNTRY/GHA/POPULATION/WORLDPOP/gha_ppp_2015_UNadj.tif
/home/public/Data/COUNTRY/GHA/POPULATION/WORLDPOP/gha_ppp_2017_UNadj.tif
/home/public/Data/COUNTRY/GHA/POPULATION/WORLDPOP/ppp_prj_2015_GHA_UNadj.tif
/home/public/Data/COUNTRY/GHA/POPULATION/WORLDPOP/ppp_prj_2017_GHA_UNadj.tif
11:43:01	***** Calculating Urban
11:43:01	***Starting 
11:43:01	/home/wb411133/temp/GHA_URBAN_DATA/FINAL_STANDARD/gha_ppp_2015_UNadj_urban.tif
11:43:01	/home/wb411133/temp/GHA_URBAN_DATA/FINAL_STANDARD/gha_ppp_2015_UNadj_urban_hd.tif
11:43:01	/home/wb411133/temp/GHA_URBAN_DATA/FINAL_STANDARD/gha_ppp_2017_UNadj_urban.tif
11: